# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [55]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.ensemble import EasyEnsembleClassifier

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [5]:
# Create our features
x_cols = [i for i in df.columns if i not in ("loan_status")]
X = df[x_cols]

# Create our target
y = df["loan_status"]

In [6]:
# Check which data is a Int or Float and which are strings
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 85 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   68817 non-null  float64
 1   int_rate                    68817 non-null  float64
 2   installment                 68817 non-null  float64
 3   home_ownership              68817 non-null  object 
 4   annual_inc                  68817 non-null  float64
 5   verification_status         68817 non-null  object 
 6   issue_d                     68817 non-null  object 
 7   pymnt_plan                  68817 non-null  object 
 8   dti                         68817 non-null  float64
 9   delinq_2yrs                 68817 non-null  float64
 10  inq_last_6mths              68817 non-null  float64
 11  open_acc                    68817 non-null  float64
 12  pub_rec                     68817 non-null  float64
 13  revol_bal                   688

In [7]:
X['debt_settlement_flag'].value_counts()

N    68817
Name: debt_settlement_flag, dtype: int64

In [8]:
# Preprocessing target values : Binary encoding using get_dummies (multiple columns), Convert homeowner to binary result
# Drop columns when there is an unique value for all rows
X.drop(columns = ['pymnt_plan', 'hardship_flag', 'debt_settlement_flag'], inplace = True)
X = pd.get_dummies(X, columns=['home_ownership', 
                               'verification_status',
                               'issue_d',
                               'initial_list_status',
                               'next_pymnt_d',
                               'application_type'])
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,verification_status_Verified,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,0,0,1,0,1,0,1,1,0
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,0,0,1,0,1,0,1,1,0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,0,0,1,0,1,0,1,1,0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,0,0,1,0,1,0,1,1,0
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,0,0,0,1,0,1,0,1,1,0


In [10]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [62]:
y_test.value_counts()

low_risk     17104
high_risk      101
Name: loan_status, dtype: int64

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [12]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [13]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
scaler_model = scaler.fit(X_train)

In [14]:
# Scale the training and testing data
X_test_scaled = scaler_model.transform(X_test)
X_train_scaled = scaler_model.transform(X_train)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [65]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500, random_state =1)
rf_model = rf_model.fit(X_train_scaled, y_train)

In [66]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.6781593559262381

In [71]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Low risk", "Actual High risk"], columns=["Predicted High risk", "Predicted Low risk"])
    
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted High risk,Predicted Low risk
Actual Low risk,36,65
Actual High risk,2,17102


In [68]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.95      0.36      1.00      0.52      0.60      0.33       101
   low_risk       1.00      1.00      0.36      1.00      0.60      0.38     17104

avg / total       1.00      1.00      0.36      1.00      0.60      0.38     17205



In [69]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_

list = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
list[:10]

[(0.07476421387837523, 'total_rec_int'),
 (0.06604322253141792, 'last_pymnt_amnt'),
 (0.06281968808771492, 'total_rec_prncp'),
 (0.06169621813013381, 'total_pymnt'),
 (0.05933575930741908, 'total_pymnt_inv'),
 (0.022251973027969486, 'installment'),
 (0.02044655502855882, 'out_prncp'),
 (0.01992277021095597, 'out_prncp_inv'),
 (0.017297829002093305, 'dti'),
 (0.016850006644768184, 'mo_sin_old_rev_tl_op')]

### Easy Ensemble Classifier

In [56]:
# Train the Classifier
classifier = EasyEnsembleClassifier(
        random_state=0)

classifier.fit(X_train_scaled, y_train.ravel())

EasyEnsembleClassifier(random_state=0)

In [57]:
# Calculated the balanced accuracy score
predictions = classifier.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions)

0.9034783711065213

In [70]:
# Display the confusion matrix
cm_easy = confusion_matrix(y_test, predictions)

cm_easy_df = pd.DataFrame(
    cm_easy, index=["Actual High risk", "Actual Low risk"], columns=["Predicted High risk", "Predicted Low risk"])
    
print("Confusion Matrix")
display(cm_easy_df)

Confusion Matrix


,Predicted High risk,Predicted Low risk
Actual High risk,90,11
Actual Low risk,1439,15665


In [59]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.06      0.89      0.92      0.11      0.90      0.81       101
   low_risk       1.00      0.92      0.89      0.96      0.90      0.82     17104

avg / total       0.99      0.92      0.89      0.95      0.90      0.82     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    EasyEnsembleClassifier has the best balance accuracy score

2. Which model had the best recall score?

    Random forest has the best recall

3. Which model had the best geometric mean score?

    RandomForest has the best geometric mean score, which is 0.581
    formula geomaetric mean score : FMI = TP / sqrt((TP + FP) * (TP + FN))
    FMI_randomForest = 0.581
    FMI_EasyEnsemble = 0.229

4. What are the top three features?

   The best features are :  total_rec_int , last_pymnt_amnt , total_pymnt_inv